## Requests and BeautifulSoup

In [26]:
#pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup # pip install beautifulsoup4 if it is not already in your environment
import requests

[requests](https://pypi.org/project/requests/) is a Python package for the HTTP-protocol. It can be applied, e.g. for getting an arbitrary web-page via it's URL. In the example below it is applied to import the german Wikipedia page of Stuttgart. The first 800 characters of the downloaded web-pages are displayed. 

In [2]:
r=requests.get("https://www.autoscout24.de/lst/mercedes-benz?sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&ocs_listing=include&search_id=1s3wv2m5o0d")

In [42]:
r.text[:800]

'<!DOCTYPE html><html lang="de-DE"><head><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><title>Mercedes-Benz Gebrauchtwagen kaufen - AutoScout24</title><link rel="preload" href="https://prod.pictures.autoscout24.net/listing-images/3cd959b6-7665-4dff-b302-e03b4ac5dad1_20985d94-5304-48df-9c03-76f15a47e56a.jpg/250x188.webp" as="image" media="(min-width: 768px) and (max-width: 1099px) and (-webkit-max-device-pixel-ratio: 1)"/><link rel="preload" href="https://prod.pictures.autoscout24.net/listing-images/3cd959b6-7665-4dff-b302-e03b4ac5dad1_20985d94-5304-48df-9c03-76f15a47e56a.jpg/480x360.webp" as="image" media="(max-width: 767px), (min-width: 1100px), (-webkit-min-device-pixel-ratio: 1.01)"/><meta name="next-head-count" content="5"/><link rel="preconnect" href="https'

As can be seen, the data returned by `requests.get()` contains the entire web-page, including all-types of markup and meta-information. In order to extract dedicated elements, e.g. images, links, text, tables etc, [Beautiful Soup (bs4)](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is the tool of choice. BS4 is a popular and comprehensive Python library for HTML- and XML-parsing.

Below, the first example demonstrates how to get all links in web-page. In the second example all tables are retained. 


In [3]:
soup=BeautifulSoup(r.text,"html.parser")

In [12]:
#bringt nix
css_soup = BeautifulSoup('<p class="cldt-summary-full-item"></p>')
css_soup.p['class']
css_soup

<html><body><p class="cldt-summary-full-item"></p></body></html>

In [5]:
#brauchen wir auch nicht
for c in soup.findAll('a'):
    if c.has_attr('href'):
        print(c)

<a href="tel:+4989444561000"> +49 89 444 56 1000</a>
<a data-track="contact" href="https://www.autoscout24.de/unternehmen/kontakt/">Kontakt</a>
<a data-track="faq" href="http://ww2.autoscout24.de/service/faq/faq_b2b.aspx">Häufige Fragen</a>
<a data-track="infoportal" href="http://ww2.autoscout24.de/partner-infoportal.aspx">Infoportal</a>
<a class="icon-auto24" data-track="logo" href="https://www.autoscout24.de"> <img loading="lazy" src="/assets/as24-header-footer/auto24-horizontal.0e6b6897.svg"/> </a>
<a data-track="cockpit" href="https://www.autoscout24.de/cockpit/">Cockpit</a>
<a data-track="search,cars" href="https://www.autoscout24.de/detailsuche?offer=U&amp;cy=D">PKW suchen</a>
<a data-track="search,motorbikes" href="https://www.autoscout24.de/detailsuche?sort=standard&amp;desc=0&amp;ustate=N%2CU&amp;cy=D&amp;atype=B">Motorräder suchen</a>
<a data-track="search,dealer" href="http://haendler.autoscout24.de/">Händler suchen</a>
<a data-track="search,caravan" href="https://www.autosc

In [4]:
import pandas as pd

In [10]:
AutoDF=pd.DataFrame(columns=["car"])
for c in soup.select("div[class^=ListItem_header]"):
    AutoDF = AutoDF.append({"car":c.text}, ignore_index=True)
AutoDF

,car
0,"Mercedes-Benz E 400 CGI Coupe AMG COMAND,PANOR..."
1,"Mercedes-Benz SLC 43 AMG COMAND KEYLESS,LEDILS..."
2,Mercedes-Benz E 53 AMG E53 AMG 4Matic Coupe Pa...
3,Mercedes-Benz SLK 200 *1.Hand*Scheckheft*Klima...
4,Mercedes-Benz S 63 AMG 4Matic Lang +1 Hand+Sch...
5,Mercedes-Benz C 63 AMG Coupé*Navi*ACC*Spur-Ass...
6,Mercedes-Benz S 350 CGI AMG 19Z. ERST:32.450KM...
7,"Mercedes-Benz CL 500 Coupe V8 / SCHIEBEDACH,LU..."
8,"Mercedes-Benz S 400 HYBRID DESIGNO KEYLESSGO,S..."
9,Mercedes-Benz E 350 4M AMG PLUS DESIGNO DISTRO...


In [11]:
price=[]
for c in soup.select("div[class^=ListItem_pricerow]"):
    price.append(c.text)
AutoDF.insert(1, column='price', value=price)

Preis müssen wir trimmen auf numerischen Wert, Einheit in Überschrift
Einschätzung ob guter Preis odern icht entweder löschen oder extra Spalte

In [12]:
VehicleDetailDF = pd.DataFrame(columns=["km", "Erstzulassung", "PS", "Zustand", "Fahrzeughalter", "Getriebe", "Kraftstoff", "Verbrauch", "Emissionen"])

for car in soup.select("div[class^=VehicleDetailTable]"):
    VehicleDetailList = []
    for c in car:
        VehicleDetailList.append(c.text)
    VehicleDetailDF = VehicleDetailDF.append({"km":VehicleDetailList[0], "Erstzulassung":VehicleDetailList[1], "PS":VehicleDetailList[2], "Zustand":VehicleDetailList[3], "Fahrzeughalter":VehicleDetailList[4], "Getriebe":VehicleDetailList[5], "Kraftstoff": VehicleDetailList[6], "Verbrauch":VehicleDetailList[7], "Emissionen":VehicleDetailList[8]}, ignore_index=True)

VehicleDetailDF

,km,Erstzulassung,PS,Zustand,Fahrzeughalter,Getriebe,Kraftstoff,Verbrauch,Emissionen
0,99.978 km,06/2015,245 kW (333 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"7,3 l/100 km (komb.)",168 g/km (komb.)
1,21.000 km,06/2016,270 kW (367 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"7,8 l/100 km (komb.)",178 g/km (komb.)
2,29.035 km,07/2019,320 kW (435 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"8,7 l/100 km (komb.)",200 g/km (komb.)
3,48.754 km,06/2011,135 kW (184 PS),Gebraucht,1 Fahrzeughalter,Schaltgetriebe,Benzin,"6,4 l/100 km (komb.)",149 g/km (komb.)
4,177.000 km,03/2016,430 kW (585 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"10,3 l/100 km (komb.)",242 g/km (komb.)
5,148.600 km,12/2017,350 kW (476 PS),Gebraucht,1 Fahrzeughalter,Automatik,Benzin,"8,6 l/100 km (komb.)",200 g/km (komb.)
6,32.450 km,06/2010,200 kW (272 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,10 l/100 km (komb.),234 g/km (komb.)
7,200.000 km,07/2003,225 kW (306 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,- (l/100 km),0 g/km (komb.)
8,60.000 km,01/2013,220 kW (299 PS),Gebraucht,- (Fahrzeughalter),Automatik,Elektro/Benzin,"8,2 l/100 km (komb.)",191 g/km (komb.)
9,77.828 km,11/2014,225 kW (306 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"8,3 l/100 km (komb.)",197 g/km (komb.)


vielleicht kW und PS trennen? Einheiten raustrimmen und in Zeilenüberschrift rein

km: Einheit raus und in Zeilenüberschrift rein

Fahrzeughalter auch nur die ZAhl behalten

Verbrauch auch EInheit raus

Emission einheit raus


In [13]:
AutoDF = AutoDF.join(VehicleDetailDF)

In [14]:
AutoDF

,car,price,km,Erstzulassung,PS,Zustand,Fahrzeughalter,Getriebe,Kraftstoff,Verbrauch,Emissionen
0,"Mercedes-Benz E 400 CGI Coupe AMG COMAND,PANOR...","€ 28.780,-Sehr guter Preis",99.978 km,06/2015,245 kW (333 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"7,3 l/100 km (komb.)",168 g/km (komb.)
1,"Mercedes-Benz SLC 43 AMG COMAND KEYLESS,LEDILS...","€ 46.880,-Sehr guter Preis",21.000 km,06/2016,270 kW (367 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"7,8 l/100 km (komb.)",178 g/km (komb.)
2,Mercedes-Benz E 53 AMG E53 AMG 4Matic Coupe Pa...,"€ 67.940,-Keine Angabe",29.035 km,07/2019,320 kW (435 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"8,7 l/100 km (komb.)",200 g/km (komb.)
3,Mercedes-Benz SLK 200 *1.Hand*Scheckheft*Klima...,"€ 20.850,-Sehr guter Preis",48.754 km,06/2011,135 kW (184 PS),Gebraucht,1 Fahrzeughalter,Schaltgetriebe,Benzin,"6,4 l/100 km (komb.)",149 g/km (komb.)
4,Mercedes-Benz S 63 AMG 4Matic Lang +1 Hand+Sch...,"€ 59.990,-Sehr guter Preis",177.000 km,03/2016,430 kW (585 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"10,3 l/100 km (komb.)",242 g/km (komb.)
5,Mercedes-Benz C 63 AMG Coupé*Navi*ACC*Spur-Ass...,"€ 51.890,-Sehr guter Preis",148.600 km,12/2017,350 kW (476 PS),Gebraucht,1 Fahrzeughalter,Automatik,Benzin,"8,6 l/100 km (komb.)",200 g/km (komb.)
6,Mercedes-Benz S 350 CGI AMG 19Z. ERST:32.450KM...,"€ 24.740,-Sehr guter Preis",32.450 km,06/2010,200 kW (272 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,10 l/100 km (komb.),234 g/km (komb.)
7,"Mercedes-Benz CL 500 Coupe V8 / SCHIEBEDACH,LU...","€ 5.900,-Sehr guter Preis",200.000 km,07/2003,225 kW (306 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,- (l/100 km),0 g/km (komb.)
8,"Mercedes-Benz S 400 HYBRID DESIGNO KEYLESSGO,S...","€ 25.880,-Keine Angabe",60.000 km,01/2013,220 kW (299 PS),Gebraucht,- (Fahrzeughalter),Automatik,Elektro/Benzin,"8,2 l/100 km (komb.)",191 g/km (komb.)
9,Mercedes-Benz E 350 4M AMG PLUS DESIGNO DISTRO...,"€ 28.880,-Guter Preis",77.828 km,11/2014,225 kW (306 PS),Gebraucht,- (Fahrzeughalter),Automatik,Benzin,"8,3 l/100 km (komb.)",197 g/km (komb.)


Automarke sollte nochmal in separate Spalte

Titel gefälltmir noch nicht irgendwie... müssen wir auch noch aufhübschen

In [20]:
#brauchen wir grad nicht, aber da stehen da noch Infos, die wir noch zusätzlich wollen`? Dann müssen wir vielleicht hier noch was rausholen

for car in soup.select("article[class^=cldt-summary-full-item]"):
    for c in car.contents:
        print(c.text)
        print("-"*20)

Gesponsert
--------------------
ZurückWeiter1 / 30Honda Civic 1.8 Lim. Sport 140PS VTEC Klima TempomatSommerreifen, Einparkhilfe Sensoren hinten, Pannenkit, E10-geeignet, Armlehne, Sportpaket, Fahrerairbag, Klimaanlage€ 7.499,-Sehr guter Preis122.686 km04/2010103 kW (140 PS)Gebraucht3 FahrzeughalterSchaltgetriebeBenzin- (l/100 km)- (g/km)
--------------------
Autohaus Klann GmbH(82)Vertrieb Gebrauchtwagen • DE-14513 Teltow+ Weitere Fahrzeuge anzeigen
--------------------
ZurückWeiter1 / 3Hyundai i30 Select 120PS ✔️ sofort verfügbar | AnhängerkupplunSofort verfügbar€ 228,84/mtl.Erhöhte Ratemtl. exkl. MwSt. € 192,30Leasingdetails hier € 0,- Anzahlung48 Monate Laufzeit10.000 km/Jahr€ 0,- Anzahlung48 Monate Laufzeit10.000 km/Jahr10 km06/202288 kW (120 PS)Gebraucht- (Fahrzeughalter)SchaltgetriebeBenzin5 l/100 km (komb.)115 g/km (komb.)
--------------------
48499 Salzbergen
--------------------
Hyundai i30 N-Line 160PS ✔️ sofort verfügbar | AnhängerkupplunSofort verfügbar€ 296,20/mtl.Faire R